### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import TerraIncognitaDataset
from plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")


Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

ti = TerraIncognitaDataset(
    data_root=CFG["datasets"]["TerraIncognita"]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = "TerraIncognita" 
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-11 04:08:26,518 | INFO | Initialized experiment directories for TerraIncognita
2025-10-11 04:08:26,518 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\TerraIncognita\logs
2025-10-11 04:08:26,518 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\TerraIncognita\checkpoints
2025-10-11 04:08:26,518 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\TerraIncognita\plots


### Setup

In [4]:
domains = CFG["datasets"]["TerraIncognita"]["domains"]
loaders = {d: {"train": ti.get_dataloader(d, train=True), "val": ti.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset="TerraIncognita")
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]


{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [5]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key="TerraIncognita",
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-10-11 04:08:47,132 | INFO | === LODO: Leaving out domain 'location_38' ===



=== LODO: Leaving out domain 'location_38' ===


Evaluating: 100%|██████████| 77/77 [00:20<00:00,  3.75it/s]
2025-10-11 04:09:35,564 | INFO | [location_38] Epoch 1/5 | Train - Loss: 1.0298, Cls: 1.0237, GRQO: 0.0061, Acc: 0.6404 | Val - Loss: 2.0119, Cls: 2.0108, GRQO: 0.0011, Acc: 0.3785
2025-10-11 04:09:35,630 | INFO | [location_38] New best val acc: 0.3785


[location_38] Epoch 1/5 | Train - Loss: 1.0298, Cls: 1.0237, GRQO: 0.0061, Acc: 0.6404 | Val - Loss: 2.0119, Cls: 2.0108, GRQO: 0.0011, Acc: 0.3785
[location_38] New best val acc: 0.3785


Evaluating: 100%|██████████| 77/77 [00:20<00:00,  3.77it/s]
2025-10-11 04:10:23,348 | INFO | [location_38] Epoch 2/5 | Train - Loss: 0.4268, Cls: 0.4243, GRQO: 0.0026, Acc: 0.8572 | Val - Loss: 3.6643, Cls: 3.6637, GRQO: 0.0006, Acc: 0.1862


...

[location_100] Epoch 3/5 | Train - Loss: 0.1976, Cls: 0.1956, GRQO: 0.0020, Acc: 0.9344 | Val - Loss: 2.1218, Cls: 2.1213, GRQO: 0.0005, Acc: 0.5001
[location_100] New best val acc: 0.5001


Evaluating: 100%|██████████| 39/39 [00:14<00:00,  2.70it/s]
2025-10-11 04:24:02,663 | INFO | [location_100] Epoch 4/5 | Train - Loss: 0.1298, Cls: 0.1281, GRQO: 0.0017, Acc: 0.9579 | Val - Loss: 2.6894, Cls: 2.6892, GRQO: 0.0002, Acc: 0.4765


[location_100] Epoch 4/5 | Train - Loss: 0.1298, Cls: 0.1281, GRQO: 0.0017, Acc: 0.9579 | Val - Loss: 2.6894, Cls: 2.6892, GRQO: 0.0002, Acc: 0.4765


Evaluating: 100%|██████████| 39/39 [00:14<00:00,  2.68it/s]
2025-10-11 04:24:50,913 | INFO | [location_100] Epoch 5/5 | Train - Loss: 0.0854, Cls: 0.0840, GRQO: 0.0014, Acc: 0.9726 | Val - Loss: 4.2394, Cls: 4.2393, GRQO: 0.0001, Acc: 0.3087
2025-10-11 04:24:50,913 | INFO | [location_100] Best Acc: 0.5001
2025-10-11 04:24:50,913 | INFO | ------------------------------------------------------------
2025-10-11 04:24:50,929 | INFO | LODO finished | Mean Acc: 0.4243 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\TerraIncognita\logs\lodo_summary_20251011_042450.json


[location_100] Epoch 5/5 | Train - Loss: 0.0854, Cls: 0.0840, GRQO: 0.0014, Acc: 0.9726 | Val - Loss: 4.2394, Cls: 4.2393, GRQO: 0.0001, Acc: 0.3087
[location_100] Best Acc: 0.5001
------------------------------------------------------------
LODO finished | Mean Acc: 0.4243
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\TerraIncognita\logs\lodo_summary_20251011_042450.json


### Baseline

In [6]:
model_name ="resnet18"
baseline_results, baseline_mean = run_baseline(
    model_name=model_name,
    CFG=CFG,
    logger=logger,
    dataset_key="TerraIncognita",
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)


2025-10-11 04:24:50,948 | INFO | Initializing ResNet baseline: resnet18
2025-10-11 04:24:51,039 | INFO | === Baseline LODO: Leaving out domain 'location_38' ===


Initializing ResNet baseline: resnet18

=== Baseline LODO: Leaving out domain 'location_38' ===


2025-10-11 04:25:38,079 | INFO | [location_38] Epoch 1/5 | Train - Loss: 0.8364, Acc: 0.7182 | Val Acc: 0.2226


[location_38] Epoch 1/5 | Train - Loss: 0.8364, Acc: 0.7182 | Val Acc: 0.2226


2025-10-11 04:26:25,651 | INFO | [location_38] Epoch 2/5 | Train - Loss: 0.2685, Acc: 0.9153 | Val Acc: 0.3980


...

[location_100] Epoch 3/5 | Train - Loss: 0.1153, Acc: 0.9658 | Val Acc: 0.4212


2025-10-11 04:41:20,459 | INFO | [location_100] Epoch 4/5 | Train - Loss: 0.0672, Acc: 0.9805 | Val Acc: 0.4423


[location_100] Epoch 4/5 | Train - Loss: 0.0672, Acc: 0.9805 | Val Acc: 0.4423


2025-10-11 04:42:27,473 | INFO | [location_100] Epoch 5/5 | Train - Loss: 0.0391, Acc: 0.9885 | Val Acc: 0.4669
2025-10-11 04:42:27,473 | INFO | [location_100] Best Val Acc: 0.4669
2025-10-11 04:42:27,474 | INFO | ------------------------------------------------------------
2025-10-11 04:42:27,474 | INFO | Baseline LODO (resnet18) finished | Mean Acc: 0.4299


[location_100] Epoch 5/5 | Train - Loss: 0.0391, Acc: 0.9885 | Val Acc: 0.4669
[location_100] Best Val Acc: 0.4669
------------------------------------------------------------
Baseline LODO (resnet18) finished | Mean Acc: 0.4299
